In [8]:
## IMPORTS

import os
import sys
import time
import math
from datetime import datetime
from io import StringIO


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy.polynomial.polynomial import Polynomial
import scipy.integrate as integrate
import scipy.optimize as sco
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import openpyxl
import networkx as nx
import sklearn.metrics
from IPython.core.display import HTML
import plotly.io as pio
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl as opxl
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
import time


sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\parallel-flow-distribution-pressure-loss\ansys')
sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\PVT-thermal-performance-model')
import jou_gen as jg
import ansys_py_bridge as apb
import ansys.fluent.core as pyfluent
import model as ty
import proc as pr
import plot_functions_here as pfun
import heat_transfer as bht
import fluids as fds
import ht
import general as gen

# Visualisation des résultats

In [19]:
folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'

hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')
panelSpecs = pr.import_geometry(geometry_path)

nb_mesh = 1
nb_cases = 1
limit_big_it = 8
nb_hx = 5

In [20]:
## Récupération des données sauvegardées

ht_tot_mesh_case_list = []
ht_rad_mesh_case_list = []
ht_conv_mesh_case_list = []
CFD_mesh_case_list = []
df_one_mesh_case_list = []
slices_df_mesh_case_list = []
PyFluent_mesh_case_list = []

for mesh in range(nb_mesh) :

    folder_mesh = f'Simu_M{mesh+1}'
    folder_path_mesh = os.path.join(folder_path, folder_mesh)

    ht_tot_case_list = []
    ht_rad_case_list = []
    ht_conv_case_list = []
    CFD_case_list = []
    df_one_case_list = []
    slices_df_case_list = []
    PyFluent_case_list = []

    for case in range(nb_cases) :
    # case = 2
    # if True :
        ## Récupération des résultats

        ht_tot_list = []
        ht_rad_list = []
        ht_conv_list = []
        CFD_list = []
        df_one_list = []
        slices_df_list = []
        PyFluent_list = []

        # Récupération des données pour chaque cas
        folder_name = f'cas_{case+1}'
        folder_path_case = os.path.join(folder_path_mesh, folder_name)

        # Définir le chemin du dossier pour les fichiers de résultats
        hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test')

        # for iteration in range(1, limit_big_it+1) :
        for iteration in range(limit_big_it) :
            file_path_result_CFD = hyp['CFD_ht_path']+f'_{iteration}.csv' 
            file_path_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{iteration}.csv'
            file_path_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{iteration}.csv'
            file_path_Inputs_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{iteration}.csv'

            file_path_df_one_cleaned = hyp['CFD_ht_path']+'_df_one_cleaned' + f'_{iteration}.csv'
            file_path_slices_df_cleaned = hyp['CFD_ht_path']+'_slices_df_cleaned' + f'_{iteration}.csv'


            df_CFD = pd.read_csv(file_path_result_CFD, sep=';')
            df_CFD['iteration'] = iteration
            df_CFD.index = [f'part{i}' for i in range(1, nb_hx+3)]
            CFD_list.append(df_CFD)

            df_one = pd.read_csv(file_path_df_one, sep=';')
            df_one['iteration'] = iteration
            df_one.index = [f'part{i}' for i in range(1, nb_hx+3)]
            df_one_list.append(df_one)

            slices_df = pd.read_csv(file_path_slices_df, sep=';')
            slices_df['iteration'] = iteration
            slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
            slices_df_list.append(slices_df)

            df_PyFluent = pd.read_csv(file_path_Inputs_PyFluent, sep=';')
            df_PyFluent['iteration'] = iteration
            PyFluent_list.append(df_PyFluent)

            ht_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{iteration}.csv'), sep=',')
            ht_tot['iteration'] = iteration
            ht_tot_list.append(ht_tot)

            ht_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{iteration}.csv'), sep=',')
            ht_rad['iteration'] = iteration
            ht_rad_list.append(ht_rad)
            
            ht_conv = pd.read_csv(os.path.join(folder_path_case,f'ht_conv_report_{iteration}.csv'), sep=';')
            ht_conv['iteration'] = iteration
            ht_conv_list.append(ht_conv)

        # if mesh = 0 :
        #     for iteration in {6, 11} :
        #         file_path_result_CFD = hyp['CFD_ht_path']+f'_{iteration}.csv' 
        #         file_path_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{iteration}.csv'
        #         file_path_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{iteration}.csv'
        #         file_path_Inputs_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{iteration}.csv'

        #         file_path_df_one_cleaned = hyp['CFD_ht_path']+'_df_one_cleaned' + f'_{iteration}.csv'
        #         file_path_slices_df_cleaned = hyp['CFD_ht_path']+'_slices_df_cleaned' + f'_{iteration}.csv'


        #         df_CFD = pd.read_csv(file_path_result_CFD, sep=';')
        #         df_CFD['iteration'] = iteration
        #         df_CFD.index = [f'part{i}' for i in range(1, nb_hx+3)]
        #         CFD_list.append(df_CFD)

        #         df_one = pd.read_csv(file_path_df_one, sep=';')
        #         df_one['iteration'] = iteration
        #         df_one.index = [f'part{i}' for i in range(1, nb_hx+3)]
        #         df_one_list.append(df_one)

        #         slices_df = pd.read_csv(file_path_slices_df, sep=';')
        #         slices_df['iteration'] = iteration
        #         slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
        #         slices_df_list.append(slices_df)

        #         df_PyFluent = pd.read_csv(file_path_Inputs_PyFluent, sep=';')
        #         df_PyFluent['iteration'] = iteration
        #         PyFluent_list.append(df_PyFluent)

        #         ht_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{iteration}.csv'), sep=',')
        #         ht_tot['iteration'] = iteration
        #         ht_tot_list.append(ht_tot)

        #         ht_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{iteration}.csv'), sep=',')
        #         ht_rad['iteration'] = iteration
        #         ht_rad_list.append(ht_rad)
                
        #         ht_conv = pd.read_csv(os.path.join(folder_path_case,f'ht_conv_report_{iteration}.csv'), sep=';')
        #         ht_conv['iteration'] = iteration
        #         ht_conv_list.append(ht_conv)

## Avec la nouvelle façon de faire la boucle on aura la dernière itération du modèle 1D 
        # 
        # file_path_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{iteration+1}.csv'
        # file_path_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{iteration+1}.csv'

        # df_one = pd.read_csv(file_path_df_one, sep=';')
        # df_one['iteration'] = iteration+1
        # df_one.index = [f'part{i}' for i in range(1, nb_hx+3)]
        # df_one_list.append(df_one)

        # slices_df = pd.read_csv(file_path_slices_df, sep=';')
        # slices_df['iteration'] = iteration+1
        # slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
        # slices_df_list.append(slices_df)

        ht_tot_case_list.append(ht_tot_list)
        ht_rad_case_list.append(ht_rad_list)
        ht_conv_case_list.append(ht_conv_list)
        CFD_case_list.append(CFD_list)
        df_one_case_list.append(df_one_list)
        slices_df_case_list.append(slices_df_list)
        PyFluent_case_list.append(PyFluent_list)

    ht_tot_mesh_case_list.append(ht_tot_case_list)
    ht_rad_mesh_case_list.append(ht_rad_case_list)
    ht_conv_mesh_case_list.append(ht_conv_case_list)
    CFD_mesh_case_list.append(CFD_case_list)
    df_one_mesh_case_list.append(df_one_case_list)
    slices_df_mesh_case_list.append(slices_df_case_list)
    PyFluent_mesh_case_list.append(PyFluent_case_list)

In [1]:
## Calcul des transferts à partir des reports // CFD
# Définir les parties pour chaque transfert
part1_tube_back = ['manifold_yu']
part2_tube_back = ['hx_bend_yu_air', 'hx_bend_yu_pv']
part3_tube_back = ['hx_flat_yu_air']
part4_tube_back = ['hx_bend_mid_air', 'hx_bend_mid_pv']
part5_tube_back = ['hx_flat_yd_air']
part6_tube_back = ['hx_bend_yd_air', 'hx_bend_yd_pv']
part7_tube_back = ['manifold_yd']

part1_top = []
part2_top = []
part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
part4_top = []
part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
part6_top = []
part7_top = []

part1_tube_fluid = ['manifold_yu']
part2_tube_fluid = ['hx_bend_yu_air', 'hx_bend_yu_pv']
part3_tube_fluid = ['hx_flat_yu_air', 'hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
part4_tube_fluid = ['hx_bend_mid_air', 'hx_bend_mid_pv']
part5_tube_fluid = ['hx_flat_yd_air', 'hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
part6_tube_fluid = ['hx_bend_yd_air', 'hx_bend_yd_pv']
part7_tube_fluid = ['manifold_yd']

component_list_tube_back = [part1_tube_back, part2_tube_back, part3_tube_back, part4_tube_back, part5_tube_back, part6_tube_back, part7_tube_back]
component_list_top = [part1_top, part2_top, part3_top, part4_top, part5_top, part6_top, part7_top]
component_list_tube_fluid = [part1_tube_fluid, part2_tube_back, part3_tube_back, part4_tube_back, part5_tube_back, part6_tube_back, part7_tube_back]

# Fonction pour calculer les Qdot pour chaque partie et chaque itération
def calculate_Qdot(mesh, case, iteration):
    ht_tot = ht_tot_mesh_case_list[mesh][case][iteration]
    ht_rad = ht_rad_mesh_case_list[mesh][case][iteration]
    ht_conv = ht_conv_mesh_case_list[mesh][case][iteration]
    
    Qdot_tube_back = []
    Qdot_top = []
    Qdot_top_rad = []
    Qdot_tube_fluid = []

    for i in range(1, nb_hx+3):
        if i == 3 or i == 5:
            Qdot_tube_back.append(4.75*ht_conv[ht_conv['Component'].isin(eval(f'part{i}_tube_back'))]['conv_ht'].sum())
            Qdot_tube_fluid.append(4.75*ht_tot[ht_tot['Component'].isin(eval(f'part{i}_tube_fluid'))]['ht'].sum())
            Qdot_top.append(4.75*ht_conv[ht_conv['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum())
            Area_part = pr.top_area_tube_contact_PV(panelSpecs[f'part{i}'])
            Area_panel = pr.top_area_tube_contact_PV(panelSpecs['main'])
            Qdot_rad = ht_rad_mesh_case_list[mesh][case][iteration][ht_rad_mesh_case_list[mesh][case][iteration]['Component'] == 'pv_front']['rad_ht'].values[0]
            Qdot_tot = ht_tot_mesh_case_list[mesh][case][iteration][ht_tot_mesh_case_list[mesh][case][iteration]['Component'] == 'pv_front']['ht'].values[0]
            Qdot_top_rad.append((Qdot_rad/Qdot_tot) * Qdot_top[-1])

        else:
            Qdot_tube_back.append(4.75*ht_conv[ht_conv['Component'].isin(eval(f'part{i}_tube_back'))]['conv_ht'].sum())
            Qdot_tube_fluid.append(-4.75*ht_tot[ht_tot['Component'].isin(eval(f'part{i}_tube_fluid'))]['ht'].sum())
            Qdot_top.append(1e-6)
            Qdot_top_rad.append(1e-6)
    
    return Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back

: 

In [369]:
Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
Qdot_top_rad
Qdot_rad = ht_rad_mesh_case_list[mesh][case][iteration][ht_rad_mesh_case_list[mesh][case][iteration]['Component'] == 'pv_front']['rad_ht'].values[0]
Qdot_tot = ht_tot_mesh_case_list[mesh][case][iteration][ht_tot_mesh_case_list[mesh][case][iteration]['Component'] == 'pv_front']['ht'].values[0]
Qdot_tot

-27.579333

In [477]:
## Ratio de transfert convectif et radiatif // CFD

def ratio_list(component_list):
    ratio_mesh_case_it_part_list = []
    for mesh in range(nb_mesh):
        ratio_case_it_list = []
        for case in range(nb_cases):
            ratio_case_it_part_list = []
            for iteration in range(limit_big_it):
                ratio_part_list = []
                for part in component_list:
                    if part == [] :
                        ratio_conv = 0
                        ratio_rad = 0
                    else :
                        conv = ht_conv_mesh_case_list[mesh][case][iteration][ht_conv_mesh_case_list[mesh][case][iteration]['Component'].isin(part)]['conv_ht'].sum()
                        tot = ht_tot_mesh_case_list[mesh][case][iteration][ht_tot_mesh_case_list[mesh][case][iteration]['Component'].isin(part)]['ht'].sum()
                        rad = ht_rad_mesh_case_list[mesh][case][iteration][ht_rad_mesh_case_list[mesh][case][iteration]['Component'].isin(part)]['rad_ht'].sum()
    
                        ratio_conv = conv / tot
                        ratio_rad = rad / tot
                    
                    ratio_part_list.append([ratio_conv, ratio_rad])
                
                ratio_case_it_part_list.append(ratio_part_list)
            ratio_case_it_list.append(ratio_case_it_part_list)
        ratio_mesh_case_it_part_list.append(ratio_case_it_list)
    
    return ratio_mesh_case_it_part_list

ratio_tube_back_list = ratio_list(component_list_tube_back)
ratio_tube_fluid_list = ratio_list(component_list_tube_fluid)

In [31]:
## Profil de température combiné mesh par cas

nb_it = 7
fig_list = []
for case in range(nb_cases) :
    fig = go.Figure()
    for mesh in range(nb_mesh) :
        # ht_tot_list, ht_rad_list, ht_conv_list, CFD_list, df_one_list, slices_df_list, PyFluent_list = get_data(plot_hyp, panelSpecs, hyp, stepConditions)
        nb_hx = int(apb.get_value('nb_hx', 'named_expression', PyFluent_mesh_case_list[0][0][0]))

        step = apb.get_value('L_1', 'named_expression', PyFluent_mesh_case_list[0][0][0]) / 100
        temperature_profiles_it = []

        cmap = px.colors.sequential.Viridis

        for iteration in range(nb_it):
            y_values_tot_iter = [0]
            T_fluid_values_tot_iter = [apb.get_value('T_fluid_in_1','named_expression',PyFluent_mesh_case_list[mesh][case][iteration])]
            temperature_profiles = []
            
            for i in range(1, nb_hx + 1):
                T_fluid_in = apb.get_value(f'T_fluid_in_{i}','named_expression',PyFluent_mesh_case_list[mesh][case][iteration])
                L = apb.get_value(f'L_{i}', 'named_expression', PyFluent_mesh_case_list[mesh][case][iteration])
                a_f = apb.get_value(f'a_f_{i}', 'named_expression', PyFluent_mesh_case_list[mesh][case][iteration])
                b_f = apb.get_value(f'b_f_{i}', 'named_expression', PyFluent_mesh_case_list[mesh][case][iteration])

                y_values = np.linspace(y_values_tot_iter[-1], y_values_tot_iter[-1] + L, int(L / step))
                if i == 3:
                    T_fluid_mean_value = apb.T_fluid_mean(T_fluid_in, L, a_f, b_f)
                    T_fluid_values = T_fluid_mean_value * np.ones(len(y_values))
                else:
                    T_fluid_values = T_fluid_fct(y_values - y_values_tot_iter[-1], T_fluid_in, a_f, b_f)

                y_values_tot_iter = np.concatenate((y_values_tot_iter, y_values[1:]))
                T_fluid_values_tot_iter = np.concatenate((T_fluid_values_tot_iter, T_fluid_values[1:]))
                temperature_profiles.append((y_values, T_fluid_values, f'part{i}'))
            
            temperature_profiles_it.append(temperature_profiles)
            color = cmap[iteration % len(cmap)]
            for y_values, T_fluid_values, part in temperature_profiles_it[iteration]:
                fig.add_trace(go.Scatter(
                    x=y_values,
                    y=T_fluid_values,
                    mode='lines',
                    name=part + f' (iteration {iteration})',
                    line=dict(color=color),
                    hovertemplate = 'y = %{y:.2f} m <br>Température = %{y:.2f} K'
                ))
        
        fig.update_layout(
            title='Profils de température pour chaque partie',
            xaxis_title='y (m)',
            yaxis_title='Température (K)',
            legend_title='Partie',
            template='plotly_white'
        )
    fig_list.append(fig)

for fig in fig_list :
    fig.show()

1D

In [493]:
## Affichage des résultats de Qdot_tube_fluid pour chaque cas // 1D pour chaque simulation

# Créer une liste de figures pour chaque cas
fig_top_list = []

# Couleurs pour chaque partie (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    for case in range(nb_cases):

        # Créer une figure pour Qdot_tube_fluid
        fig_top = go.Figure()
        
        for part in range(1, nb_hx+3):  # Supposons que vous avez 7 parties numérotées de 1 à 7
            tube_x = []
            tube_y = []
            
            for iteration in range(limit_big_it):
                # Récupérer les valeurs pour Qdot_tube_fluid pour cette partie et cette itération
                tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_fluid'].loc[f'part{part}']
    
                tube_x.append(iteration + 1)
                tube_y.append(tube_value)

            
            # Ajouter les valeurs à la figure Qdot_tube_fluid
            fig_top.add_trace(go.Scatter(x=tube_x, y=tube_y,
                                        mode='lines+markers',
                                        name=f'Partie {part}',
                                        line=dict(width=2, color=part_colors[part-1])))
        
        # Mise en forme du titre et des axes pour Qdot_tube_fluid
        fig_top.update_layout(title=f'Qdot_tube_fluid - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itérations',
                            yaxis_title='Qdot_tube_fluid')

        
        # Ajouter la figure Qdot_tube_fluid à la liste
        fig_top_list.append(fig_top)

# Afficher toutes les figures pour Qdot_tube_fluid par cas
for fig_top in fig_top_list:
    fig_top.show()


In [ ]:
## Afficher des résultats de Qdot_tube_back // 1D chaque simulation

# Définition des couleurs pour chaque partie
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Initialisation des listes de figures pour chaque simulation
fig_tube_lists = [[] for _ in range(nb_mesh)]

# Boucle sur chaque simulation (mesh)
for mesh in range(nb_mesh):
    for case in range(nb_cases):
        # Créer une figure pour Qdot_tube_back_conv
        fig_tube = go.Figure()

        for part in range(1, 8):  # Supposons que vous avez 7 parties numérotées de 1 à 7
            tube_x = []
            tube_y = []

            for iteration in range(limit_big_it):
                # Récupérer les valeurs pour Qdot_tube_back_conv pour cette partie, cette simulation et cette itération
                tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']

                tube_x.append(iteration + 1)
                tube_y.append(tube_value)

            # Ajouter les valeurs à la figure Qdot_tube_back_conv
            fig_tube.add_trace(go.Scatter(x=tube_x, y=tube_y,
                                          mode='lines+markers',
                                          name=f'Partie {part}',
                                          line=dict(width=2, color=part_colors[part-1])))

        # Mise en forme du titre et des axes pour Qdot_tube_back_conv
        fig_tube.update_layout(title=f'Qdot_tube_back - Cas {case+1} - Simu M{mesh+1}',
                               xaxis_title='Itérations',
                               yaxis_title='Qdot_tube_back [W]',
                               plot_bgcolor='rgba(0,0,0,0)')  # Fond transparent pour superposer les graphiques

        # Ajouter la figure Qdot_tube_back_conv à la liste correspondante à la simulation actuelle
        fig_tube_lists[mesh].append(fig_tube)

# Afficher toutes les figures pour Qdot_tube_back_conv par simulation
for mesh in range(nb_mesh):
    for fig_tube in fig_tube_lists[mesh]:
        fig_tube.show()


In [ ]:
## Affichage des résultats de Qdot_top_conv pour chaque cas // 1D pour chaque simulation

# Créer une liste de figures pour chaque cas
fig_top_list = []

# Couleurs pour chaque partie (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    for case in range(nb_cases):

        # Créer une figure pour Qdot_top_conv
        fig_top = go.Figure()
        
        for part in range(1, nb_hx+3):  # Supposons que vous avez 7 parties numérotées de 1 à 7
            top_x = []
            top_y = []
            
            for iteration in range(limit_big_it):
                # Récupérer les valeurs pour Qdot_top_conv pour cette partie et cette itération
                top_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
                
                tube_x.append(iteration + 1)
                tube_y.append(tube_value)
                top_x.append(iteration + 1)
                top_y.append(top_value)

            
            # Ajouter les valeurs à la figure Qdot_top_conv
            fig_top.add_trace(go.Scatter(x=top_x, y=top_y,
                                        mode='lines+markers',
                                        name=f'Partie {part}',
                                        line=dict(width=2, color=part_colors[part-1])))
        
        # Mise en forme du titre et des axes pour Qdot_top_conv
        fig_top.update_layout(title=f'Qdot_top_conv - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itérations',
                            yaxis_title='Qdot_top_conv')

        
        # Ajouter la figure Qdot_top_conv à la liste
        fig_top_list.append(fig_top)

# Afficher toutes les figures pour Qdot_top_conv par cas
for fig_top in fig_top_list:
    fig_top.show()


In [ ]:
## Affichage des résultats de Qdot_top_rad pour chaque cas // 1D pour chaque simulation

# Créer une liste de figures pour chaque cas
fig_top_list = []

# Couleurs pour chaque partie (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    for case in range(nb_cases):

        # Créer une figure pour Qdot_top_conv
        fig_top = go.Figure()
        
        for part in range(1, nb_hx+3):  # Supposons que vous avez 7 parties numérotées de 1 à 7
            top_x = []
            top_y = []
            
            for iteration in range(limit_big_it):
                # Récupérer les valeurs pour Qdot_top_conv pour cette partie et cette itération
                top_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_rad'].loc[f'part{part}']
    
                top_x.append(iteration + 1)
                top_y.append(top_value)

            
            # Ajouter les valeurs à la figure Qdot_top_conv
            fig_top.add_trace(go.Scatter(x=top_x, y=top_y,
                                        mode='lines+markers',
                                        name=f'Partie {part}',
                                        line=dict(width=2, color=part_colors[part-1])))
        
        # Mise en forme du titre et des axes pour Qdot_top_conv
        fig_top.update_layout(title=f'Qdot_top_rad - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itérations',
                            yaxis_title='Qdot_top_conv')

        
        # Ajouter la figure Qdot_top_conv à la liste
        fig_top_list.append(fig_top)

# Afficher toutes les figures pour Qdot_top_conv par cas
for fig_top in fig_top_list:
    fig_top.show()


In [ ]:
## Evolution des températures d'entrée et de sortie par itération // 1D pour chaque simulation

for mesh in range(nb_mesh):
    for case in range(nb_cases):
        data = []
        for iteration in range(limit_big_it):
            # Liste temporaire pour stocker les données de chaque partie et itération
            temp_data = []
            
            for part in range(1, nb_hx+3): 
                T_in_value = df_one_mesh_case_list[mesh][case][iteration]['T_fluid_in'].loc[f'part{part}']
                T_out_value = df_one_mesh_case_list[mesh][case][iteration]['T_fluid_out'].loc[f'part{part}']
                temp_data.append(T_in_value)
                temp_data.append(T_out_value)
            
            # Ajouter les données au tableau principal
            data.append(temp_data)

        # Noms des colonnes pour le tableau
        columns = []
        for part in range(1, nb_hx+3):
            columns.append(f'T_fluid_in_{part}')
            columns.append(f'T_fluid_out_{part}')

        # Création du DataFrame
        df_temperature = pd.DataFrame(data, columns=columns)
        df_temperature.index = [i for i in range(1, limit_big_it+1)]
        df_temperature = df_temperature.rename_axis('Itération')

        # Affichage du tableau
        print(f"Cas n°{case+1} - Simu_M{mesh+1}:")
        pr.disp_html(df_temperature)

CFD

In [490]:
## Affichage Qdot_top_conv // CFD

for mesh in range(nb_mesh):
    # Boucle sur chaque cas
    for case in range(nb_cases):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération
        Qdot_tube_values = []

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
            Qdot_tube_values.append(Qdot_top)

        
        # Création des figures pour chaque type de transfert
        fig_top = go.Figure()

        # Ajout des traces à chaque figure
        for part in range(1, nb_hx+3):
            fig_top.add_trace(go.Scatter(x=list(range(1, limit_big_it + 1)), y=[qt[part-1] for qt in Qdot_tube_values],
                                        mode='lines+markers',
                                        name=f'Part {part}',
                                        line=dict(width=2)))

        # Mise en forme du titre et des axes pour chaque figure

        fig_top.update_layout(title=f'Qdot_top - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itération',
                            yaxis_title='Qdot_top')

        # Affichage des figures
        fig_top.show()

In [ ]:
## Affichage Qdot_tube_fluid // CFD

for mesh in range(nb_mesh):
    # Boucle sur chaque cas
    for case in range(nb_cases):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération
        Qdot_tube_values = []

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
            Qdot_tube_values.append(Qdot_tube_fluid)
        
        # Création des figures pour chaque type de transfert
        fig_tube = go.Figure()

        # Ajout des traces à chaque figure
        for part in {3,5}:
            fig_tube.add_trace(go.Scatter(x=list(range(1, limit_big_it + 1)), y=[qt[part-1] for qt in Qdot_tube_values],
                                        mode='lines+markers',
                                        name=f'Part {part}',
                                        line=dict(width=2)))

        # Mise en forme du titre et des axes pour chaque figure
        fig_tube.update_layout(title=f'Qdot_tube - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itération',
                            yaxis_title='Qdot_tube')

        # Affichage des figures
        fig_tube.show()


CFD et 1D sur une même courbe

In [494]:
## Affichage Qdot_tube_back // CFD et 1D

# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    # Boucle sur chaque cas
    for case in range(nb_cases):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
        Qdot_tube_values_cfd = []
        Qdot_tube_values_1d = [[] for _ in range(nb_hx + 3)]
        Qdot_tube_values = [[] for _ in range(nb_hx + 3)]
        iterations_cfd = []
        iterations_1d = [[] for _ in range(nb_hx + 3)]
        iterations = [[] for _ in range(nb_hx + 3)]

        # Créer une figure pour les valeurs de Qdot_top
        fig_top = go.Figure()

        # Boucle sur chaque itération pour calculer les valeurs et les afficher
        for iteration in range(limit_big_it):
            # Calculer les valeurs CFD
            Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
            Qdot_tube_values_cfd.append(Qdot_tube_back)
            iterations_cfd.append(iteration + 1)

            # Récupérer les valeurs pour Qdot_top_conv pour 1D
            for part in {3,5}:
                tube_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
                Qdot_tube_values_1d[part-1].append(tube_value_1d)
                iterations_1d[part-1].append(iteration + 0.5)
                Qdot_tube_values[part-1].append(tube_value_1d)
                Qdot_tube_values[part-1].append(Qdot_tube_back[part-1])
                iterations[part-1].append(iteration + 0.5)
                iterations[part-1].append(iteration + 1)

        # Ajouter les valeurs CFD et 1D à la figure
        for part in {3,5}:
            # Tracer les valeurs CFD
            fig_top.add_trace(go.Scatter(x=iterations_cfd, 
                                        y=[qt[part-1] for qt in Qdot_tube_values_cfd],
                                        mode='markers',
                                        name=f'Part {part} CFD',
                                        line=dict(width=2, color=part_colors[part-1]),
                                        marker=dict(symbol='cross'),
                                        hovertemplate='CFD Itération: %{x}<br>Qdot_tube: %{y:.2f} W<br>Part %{text}',))

            # Tracer les valeurs 1D
            fig_top.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                        y=Qdot_tube_values_1d[part-1],
                                        mode='markers',
                                        name=f'Part {part} 1D',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                        marker=dict(symbol='x'),
                                        hovertemplate='1D Itération: %{x}<br>Qdot_tube: %{y:.2f} W<br>Part %{text}',))
            
            fig_top.add_trace(go.Scatter(x=iterations[part-1], 
                                        y=Qdot_tube_values[part-1],
                                        mode='lines',
                                        name=f'Part {part}',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot')))

        # Mise en forme du titre et des axes
        fig_top.update_layout(title=f'Qdot_tube_back - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itération',
                            yaxis_title='Qdot_tube_back [W]')

        # Affichage de la figure
        fig_top.show()

In [ ]:
## Affichage Qdot_top // CFD et 1D

# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    # Boucle sur chaque cas
    for case in range(nb_cases):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
        Qdot_top_values_cfd = []
        Qdot_top_values_1d = [[] for _ in range(nb_hx + 3)]
        Qdot_top_values = [[] for _ in range(nb_hx + 3)]
        iterations_cfd = []
        iterations_1d = [[] for _ in range(nb_hx + 3)]
        iterations = [[] for _ in range(nb_hx + 3)]

        # Créer une figure pour les valeurs de Qdot_top
        fig_top = go.Figure()

        # Boucle sur chaque itération pour calculer les valeurs et les afficher
        for iteration in range(limit_big_it):
            # Calculer les valeurs CFD
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            Qdot_top_values_cfd.append(Qdot_top)
            iterations_cfd.append(iteration + 1)

            # Récupérer les valeurs pour Qdot_top_conv pour 1D
            for part in range(1, nb_hx + 3):
                top_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
                Qdot_top_values_1d[part-1].append(top_value_1d)
                iterations_1d[part-1].append(iteration + 0.5)
                Qdot_top_values[part-1].append(top_value_1d)
                Qdot_top_values[part-1].append(Qdot_top[part-1])
                iterations[part-1].append(iteration + 0.5)
                iterations[part-1].append(iteration + 1)

        # Ajouter les valeurs CFD et 1D à la figure
        for part in range(1, nb_hx + 3):
            # Tracer les valeurs CFD
            fig_top.add_trace(go.Scatter(x=iterations_cfd, 
                                        y=[qt[part-1] for qt in Qdot_top_values_cfd],
                                        mode='markers',
                                        name=f'Part {part} CFD',
                                        line=dict(width=2, color=part_colors[part-1]),
                                        marker=dict(symbol='cross'),
                                        hovertemplate='CFD Itération: %{x}<br>Qdot_top: %{y:.2f} W<br>Part %{text}',))

            # Tracer les valeurs 1D
            fig_top.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                        y=Qdot_top_values_1d[part-1],
                                        mode='markers',
                                        name=f'Part {part} 1D',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                        marker=dict(symbol='x'), 
                                        hovertemplate='1D Itération: %{x}<br>Qdot_top: %{y:.2f} W<br>Part %{text}',))
            
            fig_top.add_trace(go.Scatter(x=iterations[part-1], 
                                        y=Qdot_top_values[part-1],
                                        mode='lines',
                                        name=f'Part {part}',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot')))

        # Mise en forme du titre et des axes
        fig_top.update_layout(title=f'Qdot_top - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itération',
                            yaxis_title='Qdot_top')

        # Affichage de la figure
        fig_top.show()


Comparaison des maillages


In [492]:
## Comparaison Qdot_tube_back_conv pour chaque partie // 1D

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_tube_back_conv pour chaque partie
        tube_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_tube_back_conv pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back_conv'].loc[f'part{part}']
            tube_values.append(tube_value)

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + mesh * bar_width - 0.4,
            y=tube_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_tube_back_conv - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_tube_back_conv',
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [ ]:
## Comparaison Qdot_top_conv pour chaque partie // 1D

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_tube_back_conv pour chaque partie
        tube_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_tube_back_conv pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
            tube_values.append(tube_value)

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + mesh * bar_width - 0.4,
            y=tube_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_top_conv - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_tube_back_conv',
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [ ]:
## Comparaison Qdot_top_rad pour chaque partie // 1D

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_tube_back_conv pour chaque partie
        tube_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_tube_back_conv pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_rad'].loc[f'part{part}']
            tube_values.append(tube_value)

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + mesh * bar_width - 0.4,
            y=tube_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_top_rad - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_tube_back_conv',
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [273]:
## Comparaison Qdot_top_rad pour chaque partie // CFD

# Couleurs pour chaque simulation (autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 3)  # positions pour chaque partie (1 à nb_hx)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_top_rad pour chaque partie
        top_rad_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_top_rad pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            Area_part = pr.top_area_tube_contact_PV(panelSpecs[f'part{part}'])
            Area_panel = pr.top_area_tube_contact_PV(panelSpecs['main'])
            Qdot_rad = ht_rad_mesh_case_list[mesh][case][iteration][ht_rad_mesh_case_list[mesh][case][iteration]['Component'] == 'pv_front']['rad_ht'].values[0]
            Qdot_part_rad = (Area_part/Area_panel) * Qdot_rad
            top_rad_values.append(Qdot_part_rad)

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + (mesh - (nb_mesh - 1) / 2) * bar_width,
            y=top_rad_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_top_rad - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_top_rad',
        xaxis=dict(
            tickmode='array',
            tickvals=np.arange(1, nb_hx + 3),
            ticktext=[f'Part {i}' for i in range(1, nb_hx + 3)]
        ),
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [489]:
## Comparaison Qdot_top_conv pour chaque partie // CFD

# Couleurs pour chaque simulation (autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 3)  # positions pour chaque partie (1 à nb_hx)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_top_rad pour chaque partie
        top_rad_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_top_rad pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
            top_rad_values.append(Qdot_top[part - 1])

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + (mesh - (nb_mesh - 1) / 2) * bar_width,
            y=top_rad_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_top - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_top',
        xaxis=dict(
            tickmode='array',
            tickvals=np.arange(1, nb_hx + 3),
            ticktext=[f'Part {i}' for i in range(1, nb_hx + 3)]
        ),
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [487]:
## Comparaison Qdot_tube_back pour chaque partie // CFD

# Couleurs pour chaque simulation (autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 3)  # positions pour chaque partie (1 à nb_hx)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_top_rad pour chaque partie
        top_rad_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_top_rad pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
            top_rad_values.append(Qdot_tube_back[part - 1])

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + (mesh - (nb_mesh - 1) / 2) * bar_width,
            y=top_rad_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_tube - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_tube',
        xaxis=dict(
            tickmode='array',
            tickvals=np.arange(1, nb_hx + 3),
            ticktext=[f'Part {i}' for i in range(1, nb_hx + 3)]
        ),
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [460]:
## Comparaison de la convergence Qdot_top_conv // CFD et 1D 

# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour les valeurs de Qdot_tube
    fig_top = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
        Qdot_top_values_cfd = []
        Qdot_top_values_1d = [[] for _ in range(nb_hx + 3)]
        Qdot_combined_values = [[] for _ in range(nb_hx + 3)]
        iterations_cfd = []
        iterations_1d = [[] for _ in range(nb_hx + 3)]
        iterations_combined = [[] for _ in range(nb_hx + 3)]

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            # Calculer les valeurs CFD
            Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
            Qdot_top_values_cfd.append(Qdot_top)
            iterations_cfd.append(iteration + 1)

            # Récupérer les valeurs pour Qdot_tube_back pour 1D
            for part in range(1, nb_hx + 3):
                top_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
                Qdot_top_values_1d[part-1].append(top_value_1d)
                iterations_1d[part-1].append(iteration + 0.5)

                # Combinaison des valeurs CFD et 1D
                Qdot_combined_values[part-1].append(top_value_1d)
                Qdot_combined_values[part-1].append(Qdot_top[part-1])
                iterations_combined[part-1].append(iteration + 0.5)
                iterations_combined[part-1].append(iteration + 1)


        # Ajouter les valeurs CFD et 1D à la figure
        for part in range(1, nb_hx + 3):
            # Tracer les valeurs CFD
            fig_top.add_trace(go.Scatter(x=iterations_cfd, 
                                          y=[qt[part-1] for qt in Qdot_top_values_cfd],
                                          mode='markers',
                                          name=f'Part {part} CFD - Simu M{mesh+1}',
                                          line=dict(width=2, color=part_colors[part-1]),
                                          marker=dict(symbol='cross'),
                                          hovertemplate=f'1D Simu M{mesh+1}<br>Itération:'+ ' %{x}<br>Qdot_top: %{y:.2f}<extra></extra>'))

            # Tracer les valeurs 1D
            fig_top.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                          y=Qdot_top_values_1d[part-1],
                                          mode='markers',
                                          name=f'Part {part} 1D - Simu M{mesh+1}',
                                          line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                          marker=dict(symbol='x'),
                                          hovertemplate=f'CFD Simu M{mesh+1}<br>Itération:'+' %{x}<br>Qdot_top: %{y:.2f}<extra></extra>'))
            
            # Tracer la courbe combinée
            fig_top.add_trace(go.Scatter(x=iterations_combined[part-1], 
                                          y=Qdot_combined_values[part-1],
                                          mode='lines',
                                          name=f'Part {part} Combiné',
                                          line=dict(width=2, color=part_colors[part-1], dash ='dot')))

    # Mise en forme du titre et des axes
    fig_top.update_layout(title=f'Qdot_top - Cas {case+1}',
                           xaxis_title='Itération',
                           yaxis_title='Qdot_tube_back',
                           legend_title='Partie et Simulation')

    # Affichage de la figure
    fig_top.show()


In [461]:
## Comparaison Qdot_tube_back pour chaque partie // CFD et 1D

# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour les valeurs de Qdot_tube
    fig_tube = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
        Qdot_tube_values_cfd = []
        Qdot_tube_values_1d = [[] for _ in range(nb_hx + 3)]
        Qdot_combined_values = [[] for _ in range(nb_hx + 3)]
        iterations_cfd = []
        iterations_1d = [[] for _ in range(nb_hx + 3)]
        iterations_combined = [[] for _ in range(nb_hx + 3)]

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            # Calculer les valeurs CFD
            Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
            Qdot_tube_values_cfd.append(Qdot_tube_back)
            iterations_cfd.append(iteration + 1)

            # Récupérer les valeurs pour Qdot_tube_back pour 1D
            for part in {3,5}:
                tube_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
                Qdot_tube_values_1d[part-1].append(tube_value_1d)
                iterations_1d[part-1].append(iteration + 0.5)

                # Combinaison des valeurs CFD et 1D
                Qdot_combined_values[part-1].append(tube_value_1d)
                Qdot_combined_values[part-1].append(Qdot_tube_back[part-1])
                iterations_combined[part-1].append(iteration + 0.5)
                iterations_combined[part-1].append(iteration + 1)
                

        # Ajouter les valeurs CFD et 1D à la figure
        for part in {3,5}:
            # Tracer les valeurs CFD
            fig_tube.add_trace(go.Scatter(x=iterations_cfd, 
                                          y=[qt[part-1] for qt in Qdot_tube_values_cfd],
                                          mode='markers',
                                          name=f'Simu M{mesh+1}-Part{part} CFD',
                                          line=dict(width=2, color=part_colors[part-1]),
                                          marker=dict(symbol='cross'),
                                          hovertemplate=f'CFD Simu M{mesh+1}<br>Itération:'+' %{x}<br>Qdot_tube_back: %{y}<extra></extra>'))

            # Tracer les valeurs 1D
            fig_tube.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                          y=Qdot_tube_values_1d[part-1],
                                          mode='markers',
                                          name=f'Simu M{mesh+1}-Part{part} 1D',
                                          line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                          marker=dict(symbol='x'),
                                          hovertemplate=f'1D Simu M{mesh+1}<br>Itération:' +' %{x}<br>Qdot_tube_back: %{y}<extra></extra>'))
            
            # Tracer la courbe combinée
            fig_tube.add_trace(go.Scatter(x=iterations_combined[part-1], 
                                          y=Qdot_combined_values[part-1],
                                          mode='lines',
                                          name=f'Part {part} Combiné',
                                          line=dict(width=2, color=part_colors[part-1], dash ='dot')))

    # Mise en forme du titre et des axes
    fig_tube.update_layout(title=f'Qdot_tube_back - Cas {case+1}',
                           xaxis_title='Itération',
                           yaxis_title='Qdot_tube_back',
                           legend_title='Partie et Simulation')

    # Affichage de la figure
    fig_tube.show()


In [462]:
## Comparaison Qdot_tube_fluid pour chaque partie // CFD et 1D

# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour les valeurs de Qdot_tube
    fig_tube = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
        Qdot_tube_values_cfd = []
        Qdot_tube_values_1d = [[] for _ in range(nb_hx + 3)]
        Qdot_combined_values = [[] for _ in range(nb_hx + 3)]
        iterations_cfd = []
        iterations_1d = [[] for _ in range(nb_hx + 3)]
        iterations_combined = [[] for _ in range(nb_hx + 3)]

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            # Calculer les valeurs CFD
            Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
            Qdot_tube_values_cfd.append(Qdot_tube_fluid)
            iterations_cfd.append(iteration + 1)

            # Récupérer les valeurs pour Qdot_tube_back pour 1D
            for part in {3,5}:
                tube_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_fluid'].loc[f'part{part}']
                Qdot_tube_values_1d[part-1].append(tube_value_1d)
                iterations_1d[part-1].append(iteration + 0.5)

                # Combinaison des valeurs CFD et 1D
                Qdot_combined_values[part-1].append(tube_value_1d)
                Qdot_combined_values[part-1].append(-1*Qdot_tube_fluid[part-1])
                iterations_combined[part-1].append(iteration + 0.5)
                iterations_combined[part-1].append(iteration + 1)
                

        # Ajouter les valeurs CFD et 1D à la figure
        for part in {3,5}:
            # Tracer les valeurs CFD
            fig_tube.add_trace(go.Scatter(x=iterations_cfd, 
                                          y=[-qt[part-1] for qt in Qdot_tube_values_cfd],
                                          mode='markers',
                                          name=f'Simu M{mesh+1}-Part{part} CFD',
                                          line=dict(width=2, color=part_colors[part-1]),
                                          marker=dict(symbol='cross'),
                                          hovertemplate=f'CFD Simu M{mesh+1}<br>Itération:'+' %{x}<br>Qdot_tube_fluid: %{y}<extra></extra>'))

            # Tracer les valeurs 1D
            fig_tube.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                          y=Qdot_tube_values_1d[part-1],
                                          mode='markers',
                                          name=f'Simu M{mesh+1}-Part{part} 1D',
                                          line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                          marker=dict(symbol='x'),
                                          hovertemplate=f'1D Simu M{mesh+1}<br>Itération:' +' %{x}<br>Qdot_tube_fluid: %{y}<extra></extra>'))
            
            # Tracer la courbe combinée
            fig_tube.add_trace(go.Scatter(x=iterations_combined[part-1], 
                                          y=Qdot_combined_values[part-1],
                                          mode='lines',
                                          name=f'Part {part} Combiné',
                                          line=dict(width=2, color=part_colors[part-1], dash ='dot')))

    # Mise en forme du titre et des axes
    fig_tube.update_layout(title=f'Qdot_tube_fluid - Cas {case+1}',
                           xaxis_title='Itération',
                           yaxis_title='Qdot_tube_fluid [W]',
                           legend_title='Partie et Simulation')

    # Affichage de la figure
    fig_tube.show()


### f(Tamb-Tmean)

In [202]:
## Comparaison Qdot_tube_fluid en fonction du delta Tamb-T_in // 1D

from scipy.stats import linregress

# Couleurs pour chaque partie
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Créer une figure pour Qdot_tube_fluid
    fig_top = go.Figure()
    
    # Pour chaque partie
    for part in {3, 5}: 
        tube_x = []
        tube_y = []
        
        # Pour chaque cas
        for case in range(nb_cases):
            tube_value = df_one_mesh_case_list[mesh][case][-1]['Qdot_tube_fluid'].loc[f'part{part}']
            T_in = stepConditions[case]['T_fluid_in0']
            T_out = apb.get_value('T_fluid_in_6', 'named_expression', PyFluent_mesh_case_list[mesh][case][-1])
            Tmean = (T_in - T_out) / 2
            delta_temp = stepConditions[case]['T_amb'] - Tmean

            tube_x.append(delta_temp)
            tube_y.append(tube_value)

        # Calculer la régression linéaire et le coefficient de corrélation
        a, b, r_value, p_value, std_err = linregress(tube_x, tube_y)
        line_y = np.array(tube_x) * a + b
        r_value = r_value ** 2
        
        # Ajouter les valeurs à la figure Qdot_tube_fluid
        fig_top.add_trace(go.Scatter(x=tube_x, y=tube_y,
                                     mode='lines+markers',
                                     name=f'Partie {part}',
                                     line=dict(width=2, color=part_colors[part-1]),
                                     hovertemplate=f'Partie {part}<br>Cas {case + 1}<br>%{{y:.2f}}'))
        
        # Ajouter la courbe de régression linéaire
        fig_top.add_trace(go.Scatter(x=tube_x, y=line_y,
                                     mode='lines',
                                     name=f'Qdot_tube_fluid = {a:.2f}*DT + {b:.2f} <br> r²={r_value:.2f}',
                                     line=dict(width=2, color=part_colors[part-1], dash='dash')))


    # Mise en forme du titre et des axes pour Qdot_tube_fluid
    fig_top.update_layout(title=f'Qdot_tube_fluid - Simu M{mesh + 1}',
                          xaxis_title='T_amb - T_mean',
                          yaxis_title='Qdot_tube_fluid',
                          legend_title='Partie et Régression Linéaire')

    # Afficher la figure
    fig_top.show()


In [ ]:
## Comparaison Qdot_top en fonction du delta Tamb-T_in // 1D

from scipy.stats import linregress

# Couleurs pour chaque partie
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Créer une figure pour Qdot_top_conv
    fig_top = go.Figure()
    
    # Pour chaque partie
    for part in {3, 5}: 
        tube_x = []
        tube_y = []
        
        # Pour chaque cas
        for case in range(nb_cases):
            tube_value = df_one_mesh_case_list[mesh][case][-1]['Qdot_top_conv'].loc[f'part{part}']
            T_in = stepConditions[case]['T_fluid_in0']
            T_out = apb.get_value('T_fluid_in_6', 'named_expression', PyFluent_mesh_case_list[mesh][case][-1])
            Tmean = (T_in - T_out) / 2
            delta_temp = stepConditions[case]['T_amb'] - Tmean

            tube_x.append(delta_temp)
            tube_y.append(tube_value)

        # Calculer la régression linéaire et le coefficient de corrélation
        a, b, r_value, p_value, std_err = linregress(tube_x, tube_y)
        line_y = np.array(tube_x) * a + b
        r_value = r_value ** 2
        
        # Ajouter les valeurs à la figure Qdot_top_conv
        fig_top.add_trace(go.Scatter(x=tube_x, y=tube_y,
                                     mode='lines+markers',
                                     name=f'Partie {part}',
                                     line=dict(width=2, color=part_colors[part-1]),
                                     hovertemplate=f'Partie {part}<br>Cas {case + 1}<br>%{{y:.2f}}'))
        
        # Ajouter la courbe de régression linéaire
        fig_top.add_trace(go.Scatter(x=tube_x, y=line_y,
                                     mode='lines',
                                     name=f'Qdot_top = {a:.2f}*DT + {b:.2f} <br> r²={r_value:.2f}',
                                     line=dict(width=2, color=part_colors[part-1], dash='dash')))


    # Mise en forme du titre et des axes pour Qdot_top_conv
    fig_top.update_layout(title=f'Qdot_top - Simu M{mesh + 1}',
                          xaxis_title='T_amb - T_mean',
                          yaxis_title='Qdot_top [W]',
                          legend_title='Partie et Régression Linéaire')

    # Afficher la figure
    fig_top.show()


### f(mdot)

In [ ]:
## Comparaison Qdot_tube_fluid en fonction du delta mdot // 1D

from scipy.stats import linregress

# Couleurs pour chaque partie
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

mesh = 0
# Créer une figure pour Qdot_tube_fluid
fig_tube = go.Figure()

# Pour chaque partie
for part in {3, 5}: 
    tube_x = []
    tube_y = []
    
    # Pour chaque cas
    for case in range(nb_cases):
        tube_value = df_one_mesh_case_list[mesh][case][-1]['Qdot_tube_fluid'].loc[f'part{part}']
        mdot = stepConditions[case]['mdot']

        tube_x.append(mdot)
        tube_y.append(tube_value)

    # Calculer la régression linéaire et le coefficient de corrélation
    a, b, r_value, p_value, std_err = linregress(tube_x, tube_y)
    line_y = np.array(tube_x) * a + b
    r_value = r_value ** 2
    
    # Ajouter les valeurs à la figure Qdot_tube_fluid
    fig_tube.add_trace(go.Scatter(x=tube_x, y=tube_y,
                                    mode='lines+markers',
                                    name=f'Partie {part}',
                                    line=dict(width=2, color=part_colors[part-1]),
                                    hovertemplate=f'Partie {part}<br>Cas {case + 1}<br>%{{y:.2f}}'))
    
    # Ajouter la courbe de régression linéaire
    fig_tube.add_trace(go.Scatter(x=tube_x, y=line_y,
                                    mode='lines',
                                    name=f'Qdot_tube_fluid = {a:.2f}*DT + {b:.2f} <br> r²={r_value:.2f}',
                                    line=dict(width=2, color=part_colors[part-1], dash='dash')))


# Mise en forme du titre et des axes pour Qdot_tube_fluid
fig_tube.update_layout(title=f'Qdot_tube_fluid - Simu M{mesh + 1}',
                        xaxis_title='mdot [kg/s]',
                        yaxis_title='Qdot_tube_fluid',
                        legend_title='Partie et Régression Linéaire')

# Afficher la figure
fig_tube.show()


### Cas par cas

In [495]:
### Comparaison cas par cas

# Définir le cas que vous voulez afficher
case = 2

# Affichage Qdot_top // CFD et 1D
# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
bar_width = 0.8 / nb_mesh
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)


## Convergence de Qdot_top pour chaque partie // CFD et 1D

fig_top = go.Figure()
# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
    Qdot_top_values_cfd = []
    Qdot_top_values_1d = [[] for _ in range(nb_hx + 3)]
    Qdot_combined_values = [[] for _ in range(nb_hx + 3)]
    iterations_cfd = []
    iterations_1d = [[] for _ in range(nb_hx + 3)]
    iterations_combined = [[] for _ in range(nb_hx + 3)]

    # Boucle sur chaque itération pour calculer les valeurs
    for iteration in range(limit_big_it):
        # Calculer les valeurs CFD
        Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
        Qdot_top_values_cfd.append(Qdot_top)
        iterations_cfd.append(iteration + 1)

        # Récupérer les valeurs pour Qdot_top pour 1D
        # for part in range(1, nb_hx + 3):
        for part in {3,5}:
            top_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
            Qdot_top_values_1d[part-1].append(top_value_1d)
            iterations_1d[part-1].append(iteration + 0.5)

            # Combinaison des valeurs CFD et 1D
            Qdot_combined_values[part-1].append(top_value_1d)
            Qdot_combined_values[part-1].append(Qdot_top[part-1])
            iterations_combined[part-1].append(iteration + 0.5)
            iterations_combined[part-1].append(iteration + 1)

    ## Avec la nouvelle méthode on enregistre 1 itération de plus pour le modèle 1D
    # for part in {3,5}:
    #     top_value_1d = df_one_mesh_case_list[mesh][case][iteration+1]['Qdot_top_conv'].loc[f'part{part}']
    #     Qdot_top_values_1d[part-1].append(top_value_1d)
    #     iterations_1d[part-1].append(iteration + 0.5)

    #     Qdot_combined_values[part-1].append(top_value_1d)
    #     iterations_combined[part-1].append(iteration + 0.5)


    # Ajouter les valeurs CFD et 1D à la figure
    # for part in range(1, nb_hx + 3):
    for part in {3,5}:
        # Tracer les valeurs CFD
        fig_top.add_trace(go.Scatter(x=iterations_cfd, 
                                        y=[qt[part-1] for qt in Qdot_top_values_cfd],
                                        mode='markers',
                                        name='Qdot_top CFD',
                                        line=dict(width=2, color=part_colors[part-1]),
                                        marker=dict(symbol='cross'),
                                        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'))

        # Tracer les valeurs 1D
        fig_top.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                        y=Qdot_top_values_1d[part-1],
                                        mode='markers',
                                        name='Qdot_top_conv 1D',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                        marker=dict(symbol='x'),
                                        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'))
        
        # Tracer la courbe combinée
        fig_top.add_trace(go.Scatter(x=iterations_combined[part-1], 
                                        y=Qdot_combined_values[part-1],
                                        mode='lines',
                                        name=f'Part {part}',
                                        line=dict(width=2, color=part_colors[part-1], dash ='dot')))

# Mise en forme du titre et des axes
fig_top.update_layout(title=f'Qdot_top - Cas {case+1}',
                        xaxis_title='Itération',
                        yaxis_title='Qdot_top [W]',
                        legend_title='Partie et Simulation')

# Affichage de la figure
fig_top.show()





## Convergence de Qdot_tube_back pour chaque partie // CFD et 1D

fig_tube = go.Figure()
# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
    Qdot_tube_values_cfd = []
    Qdot_tube_values_1d = [[] for _ in range(nb_hx + 3)]
    Qdot_combined_values = [[] for _ in range(nb_hx + 3)]
    iterations_cfd = []
    iterations_1d = [[] for _ in range(nb_hx + 3)]
    iterations_combined = [[] for _ in range(nb_hx + 3)]

    # Boucle sur chaque itération pour calculer les valeurs
    for iteration in range(limit_big_it):
        # Calculer les valeurs CFD
        Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back = calculate_Qdot(mesh, case, iteration)
        Qdot_tube_values_cfd.append(Qdot_tube_back)
        iterations_cfd.append(iteration + 1)

        # Récupérer les valeurs pour Qdot_tube_back pour 1D
        for part in {3,5}:
            tube_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
            Qdot_tube_values_1d[part-1].append(tube_value_1d)
            iterations_1d[part-1].append(iteration + 0.5)

            # Combinaison des valeurs CFD et 1D
            Qdot_combined_values[part-1].append(tube_value_1d)
            Qdot_combined_values[part-1].append(Qdot_tube_back[part-1])
            iterations_combined[part-1].append(iteration + 0.5)
            iterations_combined[part-1].append(iteration + 1)
    ## Avec la nouvelle méthode on enregistre 1 itération de plus pour le modèle 1D
    # for part in {3,5}:
    #     tube_value_1d = df_one_mesh_case_list[mesh][case][iteration+1]['Qdot_tube_back'].loc[f'part{part}']
    #     Qdot_tube_values_1d[part-1].append(tube_value_1d)
    #     iterations_1d[part-1].append(iteration + 0.5)

    #     Qdot_combined_values[part-1].append(tube_value_1d)
    #     iterations_combined[part-1].append(iteration + 0.5)

    # Ajouter les valeurs CFD et 1D à la figure
    for part in {3,5}:
        # Tracer les valeurs CFD
        fig_tube.add_trace(go.Scatter(x=iterations_cfd, 
                                        y=[qt[part-1] for qt in Qdot_tube_values_cfd],
                                        mode='markers',
                                        name='Qdot_tube_back CFD',
                                        line=dict(width=2, color=part_colors[part-1]),
                                        marker=dict(symbol='cross'),
                                        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'))

        # Tracer les valeurs 1D
        fig_tube.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                        y=Qdot_tube_values_1d[part-1],
                                        mode='markers',
                                        name='Qdot_tube_back 1D',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                        marker=dict(symbol='x'),
                                        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'))
        
        # Tracer la courbe combinée
        fig_tube.add_trace(go.Scatter(x=iterations_combined[part-1], 
                                        y=Qdot_combined_values[part-1],
                                        mode='lines',
                                        name=f'Part {part}',
                                        line=dict(width=2, color=part_colors[part-1], dash ='dot')))

# Mise en forme du titre et des axes
fig_tube.update_layout(title=f'Qdot_tube_back - Cas {case+1}',
                        xaxis_title='Itération',
                        yaxis_title='Qdot_tube_back [W]',
                        legend_title='Partie et Simulation')

# Affichage de la figure
fig_tube.show()





## Comparaison des maillages pour Qdot_tube_fluid // 1D
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_tube_fluid pour chaque partie
    tube_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_tube_fluid pour la dernière itération pour ce cas et ce maillage

## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
        iteration = limit_big_it - 1
        tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_fluid'].loc[f'part{part}']
        tube_values.append(tube_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=tube_values,
        name=f'Qdot_tube_fluid 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_tube_fluid - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_tube_fluid [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_top_conv // 1D
# Via ratio
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_top_conv pour chaque partie
    top_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_top_conv pour la dernière itération pour ce cas et ce maillage
        
## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
        iteration = limit_big_it - 1

        Area_part = pr.top_area_tube_contact_PV(panelSpecs[f'part{part}'])
        Area_panel = pr.top_area_tube_contact_PV(panelSpecs['main'])
        Qdot_rad = ht_rad_mesh_case_list[mesh][case][iteration][ht_rad_mesh_case_list[mesh][case][iteration]['Component'] == 'pv_front']['rad_ht'].values[0]
        Qdot_tot = ht_tot_mesh_case_list[mesh][case][iteration][ht_tot_mesh_case_list[mesh][case][iteration]['Component'] == 'pv_front']['ht'].values[0]
        ratio = (Area_part/Area_panel) * Qdot_rad/Qdot_tot

        top_value = (1-ratio)*df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
        top_values.append(top_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=top_values,
        name=f'Qdot_top_conv 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_top_conv - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_top_conv [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_top_rad // 1D
# Via ratio
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_top_rad pour chaque partie
    top_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_top_rad pour la dernière itération pour ce cas et ce maillage
        
## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
        iteration = limit_big_it - 1

        Area_part = pr.top_area_tube_contact_PV(panelSpecs[f'part{part}'])
        Area_panel = pr.top_area_tube_contact_PV(panelSpecs['main'])
        Qdot_rad = ht_rad_mesh_case_list[mesh][case][iteration][ht_rad_mesh_case_list[mesh][case][iteration]['Component'] == 'pv_front']['rad_ht'].values[0]
        Qdot_tot = ht_tot_mesh_case_list[mesh][case][iteration][ht_tot_mesh_case_list[mesh][case][iteration]['Component'] == 'pv_front']['ht'].values[0]
        ratio = (Area_part/Area_panel) * Qdot_rad/Qdot_tot

        top_value = ratio*df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
        top_values.append(top_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=top_values,
        name=f'Qdot_top_rad 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_top_rad - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_top_rad [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_tube_back_conv // 1D
# Via ratio
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_tube_back_conv pour chaque partie
    tube_back_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_tube_back_conv pour la dernière itération pour ce cas et ce maillage
        
## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
        iteration = limit_big_it - 1
        ratio = ratio_tube_back_list[mesh][case][iteration][part-1][0]
        tube_back_value = ratio*df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
        tube_back_values.append(tube_back_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=tube_back_values,
        name=f'Qdot_tube_back_conv 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_tube_back_conv - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_tube_back_conv (W)',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_tube_back_rad // 1D
# Via ratio
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_tube_back_rad pour chaque partie
    tube_back_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_tube_back_rad pour la dernière itération pour ce cas et ce maillage
        
## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
        iteration = limit_big_it - 1
        ratio = ratio_tube_back_list[mesh][case][iteration][part-1][1]
        tube_back_value = ratio*df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
        tube_back_values.append(tube_back_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=tube_back_values,
        name=f'Qdot_tube_back_rad 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_tube_back_rad - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_tube_back_rad [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_PV_sky // 1D
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_PV_sky pour chaque partie
    PV_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_PV_sky pour la dernière itération pour ce cas et ce maillage
        
## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
        iteration = limit_big_it - 1
        PV_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_PV_sky'].loc[f'part{part}']
        PV_values.append(PV_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=PV_values,
        name=f'Qdot_PV_sky 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_PV_sky - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_PV_sky [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()

IndexError: list index out of range

## Cas de référence

In [465]:
folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'

limit_big_it_ref = 3
nb_hx = 5
nb_method = 3

In [467]:
## Récupération des données cas de référence

## Récupération des résultats
ht_tot_AR_list = []
ht_rad_AR_list = []
ht_conv_AR_list = []
CFD_AR_list = []
df_one_AR_list = []
slices_df_AR_list = []
PyFluent_AR_list = []

# Récupération des données pour chaque cas
folder_name = f'cas_ref'
folder_path_case = os.path.join(folder_path, folder_name)

# Définir le chemin du dossier pour les fichiers de résultats
hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test')

# for iteration in range(1, limit_big_it+1) :
for iteration in range(limit_big_it_ref) :
    file_path_result_CFD = hyp['CFD_ht_path']+f'_{iteration}.csv' 
    file_path_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{iteration}.csv'
    file_path_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{iteration}.csv'
    file_path_Inputs_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{iteration}.csv'

    df_CFD = pd.read_csv(file_path_result_CFD, sep=';')
    df_CFD['iteration'] = iteration
    df_CFD.index = [f'part{i}' for i in range(1, nb_hx+3)]
    CFD_AR_list.append(df_CFD)

    df_one = pd.read_csv(file_path_df_one, sep=';')
    df_one['iteration'] = iteration
    df_one.index = [f'part{i}' for i in range(1, nb_hx+3)]
    df_one_AR_list.append(df_one)

    slices_df = pd.read_csv(file_path_slices_df, sep=';')
    slices_df['iteration'] = iteration
    slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
    slices_df_AR_list.append(slices_df)

    df_PyFluent = pd.read_csv(file_path_Inputs_PyFluent, sep=';')
    df_PyFluent['iteration'] = iteration
    PyFluent_AR_list.append(df_PyFluent)

    ht_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{iteration}.csv'), sep=',')
    ht_tot['iteration'] = iteration
    ht_tot_AR_list.append(ht_tot)

    ht_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{iteration}.csv'), sep=',')
    ht_rad['iteration'] = iteration
    ht_rad_AR_list.append(ht_rad)
    
    ht_conv = pd.read_csv(os.path.join(folder_path_case,f'ht_conv_report_{iteration}.csv'), sep=';')
    ht_conv['iteration'] = iteration
    ht_conv_AR_list.append(ht_conv)

## Avec la nouvelle façon de faire la boucle on aura la dernière itération du modèle 1D 
# 
# file_path_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{iteration+1}.csv'
# file_path_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{iteration+1}.csv'

# df_one = pd.read_csv(file_path_df_one, sep=';')
# df_one['iteration'] = iteration+1
# df_one.index = [f'part{i}' for i in range(1, nb_hx+3)]
# df_one_AR_list.append(df_one)

# slices_df = pd.read_csv(file_path_slices_df, sep=';')
# slices_df['iteration'] = iteration+1
# slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
# slices_df_AR_list.append(slices_df)



##----------------------------------------------------------------------------------------------------------------------------
## Récupération CFD uniform

iteration = 0
hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test_CFD_uniform')

file_path_result_CFD = hyp['CFD_ht_path']+f'_{iteration}.csv' 
file_path_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{iteration}.csv'
file_path_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{iteration}.csv'
file_path_Inputs_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{iteration}.csv'

CFD_uniform = pd.read_csv(file_path_result_CFD, sep=';')
CFD_uniform['iteration'] = iteration
CFD_uniform.index = [f'part{i}' for i in range(1, nb_hx+3)]


df_one_uniform = pd.read_csv(file_path_df_one, sep=';')
df_one_uniform['iteration'] = iteration
df_one_uniform.index = [f'part{i}' for i in range(1, nb_hx+3)]

slices_df_uniform = pd.read_csv(file_path_slices_df, sep=';')
slices_df_uniform['iteration'] = iteration
slices_df_uniform.index = [f'part{i}' for i in range(1, nb_hx+3)]

df_PyFluent_uniform = pd.read_csv(file_path_Inputs_PyFluent, sep=';')
df_PyFluent_uniform['iteration'] = iteration

ht_tot_uniform = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_uniform_{iteration}.csv'), sep=',')
ht_tot_uniform['iteration'] = iteration

ht_rad_uniform = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_uniform_{iteration}.csv'), sep=',')
ht_rad_uniform['iteration'] = iteration

ht_conv_uniform = pd.read_csv(os.path.join(folder_path_case,f'ht_conv_report_uniform_{iteration}.csv'), sep=';')
ht_conv_uniform['iteration'] = iteration


##----------------------------------------------------------------------------------------------------------------------------
## Récupération 1D free
iteration = 0
hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test_1D')

file_path_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{iteration}.csv'
file_path_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{iteration}.csv'
file_path_Inputs_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{iteration}.csv'

df_one_1D = pd.read_csv(file_path_df_one, sep=';')
df_one_1D['iteration'] = iteration
df_one_1D.index = [f'part{i}' for i in range(1, nb_hx+3)]

slices_df_1D = pd.read_csv(file_path_slices_df, sep=';')
slices_df_1D['iteration'] = iteration
slices_df_1D.index = [f'part{i}' for i in range(1, nb_hx+3)]

df_PyFluent_1D = pd.read_csv(file_path_Inputs_PyFluent, sep=';')
df_PyFluent_1D['iteration'] = iteration

In [468]:
## Calcul des transferts à partir des reports // CFD
# Définir les parties pour chaque transfert
part1_tube_back = ['manifold_yu']
part2_tube_back = ['hx_bend_yu_air', 'hx_bend_yu_pv']
part3_tube_back = ['hx_flat_yu_air']
part4_tube_back = ['hx_bend_mid_air', 'hx_bend_mid_pv']
part5_tube_back = ['hx_flat_yd_air']
part6_tube_back = ['hx_bend_yd_air', 'hx_bend_yd_pv']
part7_tube_back = ['manifold_yd']

part1_top = []
part2_top = []
part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
part4_top = []
part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
part6_top = []
part7_top = []

part1_tube_fluid = ['manifold_yu']
part2_tube_fluid = ['hx_bend_yu_air', 'hx_bend_yu_pv']
part3_tube_fluid = ['hx_flat_yu_air', 'hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
part4_tube_fluid = ['hx_bend_mid_air', 'hx_bend_mid_pv']
part5_tube_fluid = ['hx_flat_yd_air', 'hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
part6_tube_fluid = ['hx_bend_yd_air', 'hx_bend_yd_pv']
part7_tube_fluid = ['manifold_yd']

component_list_tube_back = [part1_tube_back, part2_tube_back, part3_tube_back, part4_tube_back, part5_tube_back, part6_tube_back, part7_tube_back]
component_list_top = [part1_top, part2_top, part3_top, part4_top, part5_top, part6_top, part7_top]
component_list_tube_fluid = [part1_tube_fluid, part2_tube_back, part3_tube_back, part4_tube_back, part5_tube_back, part6_tube_back, part7_tube_back]


Qdot_tube_back_uniform = []
Qdot_top_uniform = []
Qdot_top_rad_uniform = []
Qdot_tube_fluid_uniform = []
Qdot_PV_uniform = []

for i in range(1, nb_hx+3):
    Area_part = pr.top_area_tube_contact_PV(panelSpecs[f'part{i}'])
    Area_panel = pr.top_area_tube_contact_PV(panelSpecs['main'])
    if i == 3 or i == 5:
        
        Qdot_top_uniform.append(4.75*ht_conv_uniform[ht_conv_uniform['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum())
        Qdot_rad = ht_rad_uniform[ht_rad_uniform['Component'] == 'pv_front']['rad_ht'].values[0]
        Qdot_tot = ht_tot_uniform[ht_tot_uniform['Component'] == 'pv_front']['ht'].values[0]
        
        Qdot_top_rad_uniform.append((Qdot_rad/Qdot_tot) * Qdot_top_uniform[-1])

    else:
        Qdot_top_uniform.append(1e-6)
        Qdot_top_rad_uniform.append(1e-6)
    Qdot_tube_fluid_uniform.append(4.75*ht_tot_uniform[ht_tot_uniform['Component'].isin(eval(f'part{i}_tube_fluid') or eval(f'part_top{i}'))]['ht'].sum())
    Qdot_tube_back_uniform.append(4.75*ht_conv_uniform[ht_conv_uniform['Component'].isin(eval(f'part{i}_tube_back'))]['conv_ht'].sum())
    Qdot_PV_uniform.append((Area_part/Area_panel)*4.75*ht_rad_uniform[ht_rad_uniform['Component'].isin(['pv_front'])]['rad_ht'].sum()) ## Récupérer l'itération précédente de 1D ou en tot avec backsheet vpfront et pvhx 
    

In [469]:
Area_part = pr.top_area_tube_contact_PV(panelSpecs[f'part{3}'])
Area_panel = pr.top_area_tube_contact_PV(panelSpecs['main'])
a = (Area_part/Area_panel)*4.75*ht_rad_uniform[ht_rad_uniform['Component'].isin(['pv_front'])]['rad_ht'].sum()
a

-22.306369451576575

In [429]:
## Graphiques Qdot_tube_fluid

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_method

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Liste pour stocker les valeurs de Qdot_tube_fluid pour chaque partie
tube_values_AR = []
tube_values_uniform = []
tube_values_1D = []

# Pour chaque partie
for part in range(1, nb_hx + 3):
    # Récupérer les valeurs de Qdot_tube_fluid pour la dernière itération pour ce cas et ce maillage
    
## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
    iteration = limit_big_it_ref - 1
    tube_AR = df_one_AR_list[iteration]['Qdot_tube_fluid'].loc[f'part{part}']
    tube_values_AR.append(tube_AR)

    tube_uniform = -Qdot_tube_fluid_uniform[part-1]
    tube_values_uniform.append(tube_uniform)

    tube_value_1D = df_one_1D['Qdot_tube_fluid'].loc[f'part{part}']
    tube_values_1D.append(tube_value_1D)


# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 1 * bar_width - 0.4,
    y=tube_values_AR,
    name=f'Qdot_tube_fluid AR',
    marker_color=sim_colors[0],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode AR <br>'+'Qdot_tube_fluid = %{y:.2f} W'
))

# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 2 * bar_width - 0.4,
    y=tube_values_1D,
    name=f'Qdot_tube_fluid 1D',
    marker_color=sim_colors[1],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode 1D <br>'+'Qdot_tube_fluid = %{y:.2f} W'
))

# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 3 * bar_width - 0.4,
    y=tube_values_uniform,
    name=f'Qdot_tube_fluid CFD',
    marker_color=sim_colors[2],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode CFD uniform <br>'+'Qdot_tube_fluid = %{y:.2f} W'
))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_tube_fluid - Cas référence',
    xaxis_title='Parties',
    yaxis_title='Qdot_tube_fluid [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Méthode'
)

# Affichage de la figure
fig_comparison.show()

In [414]:
## Graphiques Qdot_top

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_method

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Liste pour stocker les valeurs de Qdot_top pour chaque partie
top_values_AR = []
top_values_uniform = []
top_values_1D = []

# Pour chaque partie
for part in range(1, nb_hx + 3):
    # Récupérer les valeurs de Qdot_top pour la dernière itération pour ce cas et ce maillage
    
## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
    iteration = limit_big_it_ref - 1
    top_AR = df_one_AR_list[iteration]['Qdot_top_conv'].loc[f'part{part}']
    top_values_AR.append(top_AR)

    top_uniform = Qdot_top_uniform[part-1]
    top_values_uniform.append(top_uniform)

    top_value_1D = df_one_1D['Qdot_top_conv'].loc[f'part{part}']
    top_values_1D.append(top_value_1D)


# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 1 * bar_width - 0.4,
    y=top_values_AR,
    name=f'Qdot_top AR',
    marker_color=sim_colors[0],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode AR <br>'+'Qdot_top = %{y:.2f} W'
))

# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 2 * bar_width - 0.4,
    y=top_values_1D,
    name=f'Qdot_top 1D',
    marker_color=sim_colors[1],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode 1D <br>'+'Qdot_top = %{y:.2f} W'
))

# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 3 * bar_width - 0.4,
    y=top_values_uniform,
    name=f'Qdot_top CFD',
    marker_color=sim_colors[2],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode CFD uniform <br>'+'Qdot_top = %{y:.2f} W'
))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_top - Cas référence',
    xaxis_title='Parties',
    yaxis_title='Qdot_top [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Méthode'
)

# Affichage de la figure
fig_comparison.show()

In [415]:
## Graphiques Qdot_tube_back

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_method

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Liste pour stocker les valeurs de Qdot_tube_back pour chaque partie
tube_back_values_AR = []
tube_back_values_uniform = []
tube_back_values_1D = []

# Pour chaque partie
for part in range(1, nb_hx + 3):
    # Récupérer les valeurs de Qdot_tube_back pour la dernière itération pour ce cas et ce maillage
    
## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
    iteration = limit_big_it_ref - 1
    tube_back_AR = df_one_AR_list[iteration]['Qdot_tube_back'].loc[f'part{part}']
    tube_back_values_AR.append(tube_back_AR)

    tube_back_uniform = Qdot_tube_back_uniform[part-1]
    tube_back_values_uniform.append(tube_back_uniform)

    tube_back_value_1D = df_one_1D['Qdot_tube_back'].loc[f'part{part}']
    tube_back_values_1D.append(tube_back_value_1D)


# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 1 * bar_width - 0.4,
    y=tube_back_values_AR,
    name=f'Qdot_tube_back AR',
    marker_color=sim_colors[0],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode AR <br>'+'Qdot_tube_back = %{y:.2f} W'
))

# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 2 * bar_width - 0.4,
    y=tube_back_values_1D,
    name=f'Qdot_tube_back 1D',
    marker_color=sim_colors[1],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode 1D <br>'+'Qdot_tube_back = %{y:.2f} W'
))

# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 3 * bar_width - 0.4,
    y=tube_back_values_uniform,
    name=f'Qdot_tube_back CFD',
    marker_color=sim_colors[2],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode CFD uniform <br>'+'Qdot_tube_back = %{y:.2f} W'
))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_tube_back - Cas référence',
    xaxis_title='Parties',
    yaxis_title='Qdot_tube_back [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Méthode'
)

# Affichage de la figure
fig_comparison.show()

In [432]:
## Graphiques Qdot_PV_sky

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_method

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Liste pour stocker les valeurs de Qdot_PV_sky pour chaque partie
PV_values_AR = []
PV_values_uniform = []
PV_values_1D = []

# Pour chaque partie
for part in range(1, nb_hx + 3):
    # Récupérer les valeurs de Qdot_PV_sky pour la dernière itération pour ce cas et ce maillage
    
## Dans la nouvelle méthode de calcul on a une itération de plus // il faudra enlever le -1
    iteration = limit_big_it_ref - 1
    PV_AR = df_one_AR_list[iteration]['Qdot_PV_sky'].loc[f'part{part}']
    PV_values_AR.append(PV_AR)

    PV_uniform = Qdot_PV_uniform[part-1]
    PV_values_uniform.append(PV_uniform)

    PV_value_1D = df_one_1D['Qdot_PV_sky'].loc[f'part{part}']
    PV_values_1D.append(PV_value_1D)


# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 1 * bar_width - 0.4,
    y=PV_values_AR,
    name=f'Qdot_PV_sky AR',
    marker_color=sim_colors[0],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode AR <br>'+'Qdot_PV_sky = %{y:.2f} W'
))

# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 2 * bar_width - 0.4,
    y=PV_values_1D,
    name=f'Qdot_PV_sky 1D',
    marker_color=sim_colors[1],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode 1D <br>'+'Qdot_PV_sky = %{y:.2f} W'
))

# Ajouter les barres pour chaque partie
fig_comparison.add_trace(go.Bar(
    x=bar_positions + 3 * bar_width - 0.4,
    y=PV_values_uniform,
    name=f'Qdot_PV_sky CFD',
    marker_color=sim_colors[2],
    width=bar_width,
    opacity=0.8,
    hovertemplate= f'Méthode CFD uniform <br>'+'Qdot_PV_sky = %{y:.2f} W'
))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_PV_sky - Cas référence',
    xaxis_title='Parties',
    yaxis_title='Qdot_tube_back [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Méthode'
)

# Affichage de la figure
fig_comparison.show()

In [ ]:
## Profil de température

import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Définir les fonctions nécessaires
def T_fluid_fct(y_values, T_fluid_in, a_f, b_f):
    return (T_fluid_in + b_f / a_f) * np.exp(a_f * y_values) - b_f / a_f

# Initialisation
step = apb.get_value('L_1', 'named_expression', PyFluent_mesh_case_list[0][0][0]) / 100
temperature_profiles_it = []

# Calcul des profils de température pour chaque itération
for iteration in range(limit_big_it):
    y_values_tot_iter = [0]
    T_fluid_values_tot_iter = [apb.get_value('T_fluid_in_1','named_expression',PyFluent_mesh_case_list[0][0][iteration])]
    temperature_profiles = []
    
    for i in range(1, nb_hx + 1):
        T_fluid_in = apb.get_value(f'T_fluid_in_{i}','named_expression',PyFluent_mesh_case_list[0][0][iteration]) # PyFluent_mesh_case_list[0][0][iteration].loc[PyFluent_mesh_case_list[0][0][iteration]['named_expression'] == f'T_fluid_in_{i}', 'value'].values[0]
        L = apb.get_value(f'L_{i}', 'named_expression', PyFluent_mesh_case_list[0][0][iteration])
        a_f = apb.get_value(f'a_f_{i}', 'named_expression', PyFluent_mesh_case_list[0][0][iteration])
        b_f = apb.get_value(f'b_f_{i}', 'named_expression', PyFluent_mesh_case_list[0][0][iteration])
        
        # Discrétisation et calcul des valeurs de température
        y_values = np.linspace(y_values_tot_iter[-1], y_values_tot_iter[-1] + L, int(L / step))
        if i == 3:
            T_fluid_mean_value = apb.T_fluid_mean(T_fluid_in, L, a_f, b_f)
            T_fluid_values = T_fluid_mean_value * np.ones(len(y_values))
        else:
            T_fluid_values = T_fluid_fct(y_values - y_values_tot_iter[-1], T_fluid_in, a_f, b_f)

        y_values_tot_iter = np.concatenate((y_values_tot_iter, y_values[1:]))
        T_fluid_values_tot_iter = np.concatenate((T_fluid_values_tot_iter, T_fluid_values[1:]))
        temperature_profiles.append((y_values, T_fluid_values, f'part{i}'))
    
    temperature_profiles_it.append(temperature_profiles)

# Tracé des courbes
cmap = px.colors.sequential.Viridis

fig = go.Figure()

for iteration in range(limit_big_it):
    color = cmap[iteration % len(cmap)]
    for y_values, T_fluid_values, part in temperature_profiles_it[iteration]:
        fig.add_trace(go.Scatter(
            x=y_values,
            y=T_fluid_values,
            mode='lines',
            name=part + f' (iteration {iteration})',
            line=dict(color=color),
            hovertemplate = 'y = %{y:.2f} m <br>Température = %{y:.2f} K'
        ))

fig.update_layout(
    title='Profils de température pour chaque partie',
    xaxis_title='y (m)',
    yaxis_title='Température (K)',
    legend_title='Partie',
    template='plotly_white'
)

fig.show()
